In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [3]:
leads = pd.read_csv("data/leads_smartmarket.csv", parse_dates=["date"])
leads.head()

,lead_id,date,channel,device
0,10000,2025-09-23,Facebook Ads,Mobile
1,10001,2025-09-04,Instagram Ads,Mobile
2,10002,2025-09-18,LinkedIn,Desktop
3,10003,2025-09-05,LinkedIn,Mobile
4,10004,2025-09-16,Emailing,Desktop


In [4]:
with open("data/campaign_smartmarket.json", "r", encoding="utf-8") as f:
    campaign_data = json.load(f)

campaigns = pd.DataFrame(campaign_data)
campaigns.head()

,campaign_id,channel,cost,impressions,clicks,conversions
0,CAMP01,Emailing,1612,47883,1470,82
1,CAMP02,Facebook Ads,3240,114188,2315,140
2,CAMP03,LinkedIn,6704,107834,2224,174
3,CAMP04,Instagram Ads,3676,149599,5023,352
4,CAMP05,Emailing,1628,95923,2269,146


In [5]:
crm = pd.read_excel("data/crm_smartmarket.xlsx")
crm.head()

,lead_id,company_size,sector,region,status
0,10000,100-500,Education,PAC,MQL
1,10001,1-10,Finance,Hauts-de-France,MQL
2,10002,500-1000,Retail,Pays de la Loire,MQL
3,10003,1-10,Industry,PAC,SQL
4,10004,1-10,Finance,IdF,MQL


In [6]:
def clean_str(s):
    return (s.astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True))

leads["channel"] = clean_str(leads["channel"])
leads["device"]  = clean_str(leads["device"])

campaigns["channel"] = clean_str(campaigns["channel"])
crm["sector"] = clean_str(crm["sector"])
crm["region"] = clean_str(crm["region"])
crm["status"] = clean_str(crm["status"])
crm["company_size"] = clean_str(crm["company_size"])

for df in [leads, crm]:
    df["lead_id"] = pd.to_numeric(df["lead_id"], errors="coerce").astype("Int64")

In [7]:
start = pd.Timestamp("2025-09-01")
end   = pd.Timestamp("2025-10-01")

leads_scope = leads[(leads["date"] >= start) & (leads["date"] < end)].copy()
leads_scope.shape

(5000, 4)

In [8]:
leads["date"].min(), leads["date"].max()

(Timestamp('2025-09-01 00:00:00'), Timestamp('2025-09-30 00:00:00'))

In [9]:
channels_campaign = set(campaigns["channel"].unique())
leads_scope = leads_scope[leads_scope["channel"].isin(channels_campaign)].copy()

channels_leads = set(leads_scope["channel"].unique())
channels_campaign, channels_leads

({'Emailing', 'Facebook Ads', 'Instagram Ads', 'LinkedIn'},
 {'Emailing', 'Facebook Ads', 'Instagram Ads', 'LinkedIn'})

In [10]:
dup_rows = leads_scope.duplicated().sum()
dup_ids  = leads_scope.duplicated(subset=["lead_id"]).sum()

dup_rows, dup_ids

(np.int64(0), np.int64(0))

In [11]:
leads_scope = leads_scope.drop_duplicates()

leads_scope = leads_scope.sort_values("date").drop_duplicates(subset=["lead_id"], keep="last")

In [12]:
leads_scope.isna().mean().sort_values(ascending=False)

lead_id    0.0
date       0.0
channel    0.0
device     0.0
dtype: float64

In [13]:
leads_scope["device"].value_counts(dropna=False)
leads_scope["channel"].value_counts(dropna=False)

channel
Facebook Ads     1461
Emailing         1428
Instagram Ads    1189
LinkedIn          922
Name: count, dtype: int64

In [14]:
df = leads_scope.merge(crm, on="lead_id", how="left", validate="one_to_one")
df.head()

,lead_id,date,channel,device,company_size,sector,region,status
0,14046,2025-09-01,Facebook Ads,Mobile,1-10,Tech,PAC,SQL
1,10527,2025-09-01,Emailing,Mobile,10-50,Education,Hauts-de-France,MQL
2,14462,2025-09-01,Instagram Ads,Desktop,10-50,Media,Nouvelle-Aquitaine,MQL
3,13234,2025-09-01,Facebook Ads,Mobile,1000+,Health,Bretagne,SQL
4,10498,2025-09-01,Instagram Ads,Mobile,100-500,Retail,Occitanie,SQL


In [15]:
df["company_size"].isna().mean()

np.float64(0.0)

In [16]:
camp = campaigns.copy()

camp["ctr"] = camp["clicks"] / camp["impressions"]
camp["conv_rate"] = camp["conversions"] / camp["clicks"]
camp["cpc"] = camp["cost"] / camp["clicks"]
camp["cpa"] = camp["cost"] / camp["conversions"]
camp

,campaign_id,channel,cost,impressions,clicks,conversions,ctr,conv_rate,cpc,cpa
0,CAMP01,Emailing,1612,47883,1470,82,0.030700,0.055782,1.096599,19.658537
1,CAMP02,Facebook Ads,3240,114188,2315,140,0.020274,0.060475,1.399568,23.142857
2,CAMP03,LinkedIn,6704,107834,2224,174,0.020624,0.078237,3.014388,38.528736
3,CAMP04,Instagram Ads,3676,149599,5023,352,0.033576,0.070078,0.731834,10.443182
4,CAMP05,Emailing,1628,95923,2269,146,0.023654,0.064346,0.717497,11.150685
5,CAMP06,Facebook Ads,1360,45063,753,8,0.016710,0.010624,1.806109,170.000000
6,CAMP07,LinkedIn,6013,92501,1455,111,0.015730,0.076289,4.132646,54.171171
7,CAMP08,Instagram Ads,1113,44939,1118,72,0.024878,0.064401,0.995528,15.458333
8,CAMP09,Emailing,4439,98406,2212,203,0.022478,0.091772,2.006781,21.866995
9,CAMP10,Facebook Ads,4770,105285,2382,206,0.022624,0.086482,2.002519,23.155340


In [17]:
camp["ctr"] = np.where(camp["impressions"] > 0, camp["clicks"] / camp["impressions"], np.nan)
camp["conv_rate"] = np.where(camp["clicks"] > 0, camp["conversions"] / camp["clicks"], np.nan)

In [18]:
leads_by_channel = df.groupby("channel")["lead_id"].nunique().reset_index(name="n_leads")

camp2 = camp.merge(leads_by_channel, on="channel", how="left")
camp2["cost_per_lead"] = camp2["cost"] / camp2["n_leads"]
camp2

,campaign_id,channel,cost,impressions,clicks,conversions,ctr,conv_rate,cpc,cpa,n_leads,cost_per_lead
0,CAMP01,Emailing,1612,47883,1470,82,0.030700,0.055782,1.096599,19.658537,1428,1.128852
1,CAMP02,Facebook Ads,3240,114188,2315,140,0.020274,0.060475,1.399568,23.142857,1461,2.217659
2,CAMP03,LinkedIn,6704,107834,2224,174,0.020624,0.078237,3.014388,38.528736,922,7.271150
3,CAMP04,Instagram Ads,3676,149599,5023,352,0.033576,0.070078,0.731834,10.443182,1189,3.091674
4,CAMP05,Emailing,1628,95923,2269,146,0.023654,0.064346,0.717497,11.150685,1428,1.140056
5,CAMP06,Facebook Ads,1360,45063,753,8,0.016710,0.010624,1.806109,170.000000,1461,0.930869
6,CAMP07,LinkedIn,6013,92501,1455,111,0.015730,0.076289,4.132646,54.171171,922,6.521692
7,CAMP08,Instagram Ads,1113,44939,1118,72,0.024878,0.064401,0.995528,15.458333,1189,0.936081
8,CAMP09,Emailing,4439,98406,2212,203,0.022478,0.091772,2.006781,21.866995,1428,3.108543
9,CAMP10,Facebook Ads,4770,105285,2382,206,0.022624,0.086482,2.002519,23.155340,1461,3.264887


In [19]:
cols_leads = [
    "lead_id", "date", "channel", "device",
    "company_size", "sector", "region", "status"
]
df_final = df[cols_leads].copy()
df_final.head()

,lead_id,date,channel,device,company_size,sector,region,status
0,14046,2025-09-01,Facebook Ads,Mobile,1-10,Tech,PAC,SQL
1,10527,2025-09-01,Emailing,Mobile,10-50,Education,Hauts-de-France,MQL
2,14462,2025-09-01,Instagram Ads,Desktop,10-50,Media,Nouvelle-Aquitaine,MQL
3,13234,2025-09-01,Facebook Ads,Mobile,1000+,Health,Bretagne,SQL
4,10498,2025-09-01,Instagram Ads,Mobile,100-500,Retail,Occitanie,SQL


In [20]:
cols_camp = [
    "campaign_id", "channel", "cost", "impressions", "clicks", "conversions",
    "ctr", "conv_rate", "cpc", "cpa", "n_leads", "cost_per_lead"
]
camp_final = camp2[cols_camp].copy()
camp_final

,campaign_id,channel,cost,impressions,clicks,conversions,ctr,conv_rate,cpc,cpa,n_leads,cost_per_lead
0,CAMP01,Emailing,1612,47883,1470,82,0.030700,0.055782,1.096599,19.658537,1428,1.128852
1,CAMP02,Facebook Ads,3240,114188,2315,140,0.020274,0.060475,1.399568,23.142857,1461,2.217659
2,CAMP03,LinkedIn,6704,107834,2224,174,0.020624,0.078237,3.014388,38.528736,922,7.271150
3,CAMP04,Instagram Ads,3676,149599,5023,352,0.033576,0.070078,0.731834,10.443182,1189,3.091674
4,CAMP05,Emailing,1628,95923,2269,146,0.023654,0.064346,0.717497,11.150685,1428,1.140056
5,CAMP06,Facebook Ads,1360,45063,753,8,0.016710,0.010624,1.806109,170.000000,1461,0.930869
6,CAMP07,LinkedIn,6013,92501,1455,111,0.015730,0.076289,4.132646,54.171171,922,6.521692
7,CAMP08,Instagram Ads,1113,44939,1118,72,0.024878,0.064401,0.995528,15.458333,1189,0.936081
8,CAMP09,Emailing,4439,98406,2212,203,0.022478,0.091772,2.006781,21.866995,1428,3.108543
9,CAMP10,Facebook Ads,4770,105285,2382,206,0.022624,0.086482,2.002519,23.155340,1461,3.264887
